In [2]:
from dotenv import load_dotenv
from metagame import TradingClient
import os

load_dotenv()

client = TradingClient(
    api_url=os.environ["API_URL"],
    jwt=os.environ["JWT"],
    act_as=int(os.environ["ACT_AS"]),
)

In [3]:
import pandas as pd

def get_market_trades(market_name: str):
    state = client.state()
    market_id = state.market_name_to_id.get(market_name)    
    if market_id is None:
        return pd.DataFrame()

    if not state.markets[market_id].hasFullTradeHistory:
        client.get_full_trade_history(market_id)
        state = client.state()

    trades = state.markets[market_id].trades
    if not trades:
        return pd.DataFrame()
    df = pd.DataFrame(trades).set_index("id")
    df["buyer"] = df["buyer_id"].map(lambda x: state.accounts[x].name)
    df["seller"] = df["seller_id"].map(lambda x: state.accounts[x].name)
    df["timestamp"] = df["transaction_id"].map(lambda x: state.transactions[x])
    return df.drop(columns=["market_id", "buyer_id", "seller_id", "transaction_id"])


In [4]:
def get_market_positions(market_name: str):
    df = get_market_trades(market_name)
    if df.empty:
        return pd.DataFrame()
    return pd.concat([
        df.groupby("buyer")["size"].sum(),
        -df.groupby("seller")["size"].sum()
    ]).groupby(level=0).sum()

def get_multiple_market_positions(market_names: list[str]):
    return pd.DataFrame({market: get_market_positions(market) for market in market_names}).fillna(0)

In [7]:
import ipywidgets as widgets

def get_dice(refresh):
    markets = ["avg_hack_b"]
    return get_multiple_market_positions(markets)

widgets.interact(get_dice, refresh=widgets.Checkbox(description="Refresh Data"))

interactive(children=(Checkbox(value=False, description='Refresh Data'), Output()), _dom_classes=('widget-inte…

<function __main__.get_dice(refresh)>

In [15]:
import random
import numpy as np

def fair_values():
    df = pd.DataFrame()

    for idx in range(1,21):
        avgs = []
        mins = []
        maxs = []
        
        # Perform 10000 simulations
        for _ in range(10000):
            rolls = [random.randint(1, 21) for _ in range(4)]
            rolls.append(idx)
            avgs.append(np.mean(rolls))
            mins.append(min(rolls))
            maxs.append(max(rolls))

        # Calculate averages
        df.at[idx, 'expected_avg'] = np.mean(avgs)
        df.at[idx, 'expected_min'] = np.mean(mins)
        df.at[idx, 'expected_max'] = np.mean(maxs)
    return df    

widgets.interact(fair_values, refresh=widgets.Checkbox(description="Refresh Data"))


interactive(children=(Output(),), _dom_classes=('widget-interact',))

<function __main__.fair_values()>